In [11]:
# !pip install jupyter ipywidgets allennlp_models textattack[tensorflow]

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import os.path
import torch
import torch.optim as optim
# from allennlp.data.dataset_readers.stanford_sentiment_tree_bank import \
#     StanfordSentimentTreeBankDatasetReader
from reader_new import StanfordSentimentTreeBankDatasetReader_NEW
# from allennlp.data.iterators import BucketIterator, BasicIterator
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.embedding import _read_pretrained_embeddings_file
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.training.trainer import Trainer
from allennlp.common.util import lazy_groups_of
from allennlp.data.token_indexers import SingleIdTokenIndexer
import pandas as pd

2022-05-12 14:19:03.577130: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
class LstmClassifier(Model):
    def __init__(self, word_embeddings, encoder, vocab):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.linear = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, tokens, label):
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        encoder_out = self.encoder(embeddings, mask)
        logits = self.linear(encoder_out)
        output = {"logits": logits}
        if label is not None:
            self.accuracy(logits, label)
            output["loss"] = self.loss_function(logits, label)
        return output

    def get_metrics(self, reset=False):
        return {'accuracy': self.accuracy.get_metric(reset)}

In [4]:
# load the binary SST dataset.
single_id_indexer = SingleIdTokenIndexer(lowercase_tokens=True) # word tokenizer

# use_subtrees gives us a bit of extra data by breaking down each example into sub sentences.
# reader = StanfordSentimentTreeBankDatasetReader_NEW(granularity="2-class",
#                                                 token_indexers={"tokens": single_id_indexer},
#                                                 use_subtrees=True)
# train_data = reader.read('./data/train.txt')
#     print(train_data)
reader = StanfordSentimentTreeBankDatasetReader_NEW(granularity="2-class",
                                                token_indexers={"tokens": single_id_indexer})
dev_data = reader.read('./data/dev.txt')

In [5]:
vocab_path = "./lstm_main_sst_model/w2v_" + "vocab"
vocab = Vocabulary.from_files(vocab_path)

In [6]:
# vocab = Vocabulary.from_instances(train_data)
# print(vocab)

In [7]:
embedding_path = "./data/crawl-300d-2M.vec.zip"
weight = _read_pretrained_embeddings_file(embedding_path,
                                          embedding_dim=300,
                                          vocab=vocab,
                                          namespace="tokens")
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=300,
                            weight=weight,
                            trainable=False)
word_embedding_dim = 300
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})


encoder = PytorchSeq2VecWrapper(torch.nn.LSTM(word_embedding_dim,
                                              hidden_size=512,
                                              num_layers=2,
                                              batch_first=True))

model = LstmClassifier(word_embeddings, encoder, vocab)
model_path = "./lstm_main_sst_model/w2v_model.th"
    
with open(model_path, 'rb') as f:
    model.load_state_dict(torch.load(f,map_location='cpu'))

  0%|          | 0/1999995 [00:00<?, ?it/s]

In [7]:
# embedding_path = "./data/crawl-300d-2M.vec.zip"
# weight = _read_pretrained_embeddings_file(embedding_path,
#                                           embedding_dim=300,
#                                           vocab=vocab,
#                                           namespace="tokens")
# token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
#                             embedding_dim=300,
#                             weight=weight,
#                             trainable=False)
# word_embedding_dim = 300
# word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})


# encoder = PytorchSeq2VecWrapper(torch.nn.LSTM(word_embedding_dim,
#                                               hidden_size=512,
#                                               num_layers=2,
#                                               batch_first=True))

# # Initialize model, cuda(), and optimizer
# model = LstmClassifier(word_embeddings, encoder, vocab)
# model.cuda()

In [9]:
iterator = BucketIterator(batch_size=32, sorting_keys=[("tokens", "num_tokens")])

iterator.index_with(vocab)

optimizer = optim.Adam(model.parameters())

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_data,
                  validation_dataset=dev_data,
                  num_epochs=4,
                  patience=1,
                  cuda_device=0)

In [ ]:
trainer.train()

In [ ]:
# where to save the model
model_path = "lstm_main_sst_model/w2v_model.th"
vocab_path = "lstm_main_sst_model/w2v_vocab"

with open(model_path, 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files(vocab_path)

In [10]:
print(model)

LstmClassifier(
  (word_embeddings): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (encoder): PytorchSeq2VecWrapper(
    (_module): LSTM(300, 512, num_layers=2, batch_first=True)
  )
  (linear): Linear(in_features=512, out_features=2, bias=True)
  (loss_function): CrossEntropyLoss()
)


In [19]:
state = torch.load('lstm_main_sst_model/w2v_model.th')
model.load_state_dict(state)

<All keys matched successfully>

In [8]:
import torch.quantization
import torch.nn as nn

In [9]:
model.cpu()
quantized_model = torch.quantization.quantize_dynamic(
    model,{nn.Linear,nn.LSTM}, dtype=torch.qint8
)

In [10]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (MB):', size/1e6)
    os.remove('temp.p')
    return size

In [11]:
model_size = print_size_of_model(model,"fp32")
quantized_model_size = print_size_of_model(quantized_model,"int8")

print("{0:.2f} times smaller".format(model_size/quantized_model_size))

model:  fp32  	 Size (MB): 34.727044
model:  int8  	 Size (MB): 23.44499
1.48 times smaller


In [12]:
quantized_model

LstmClassifier(
  (word_embeddings): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (encoder): PytorchSeq2VecWrapper(
    (_module): DynamicQuantizedLSTM(300, 512, num_layers=2, batch_first=True)
  )
  (linear): DynamicQuantizedLinear(in_features=512, out_features=2, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (loss_function): CrossEntropyLoss()
)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack.attacker import Attacker

2022-05-15 05:03:43.291502: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from allennlp.predictors import Predictor
import allennlp_models.classification
import torch.nn as nn
import textattack
import torch

class AllenNLPModel(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/sst-2-basic-classifier-glove-2019.06.27.tar.gz")
        self.model = self.predictor._model
        self.tokenizer = self.predictor._dataset_reader._tokenizer

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]

class AllenNLPModelCompressed(textattack.models.wrappers.ModelWrapper):
    def __init__(self):
        self.predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/sst-2-basic-classifier-glove-2019.06.27.tar.gz")
        model = self.predictor._model
        model.cpu()
        quantized_model = torch.quantization.quantize_dynamic(
            model,{nn.Linear,nn.LSTM}, dtype=torch.qint8
        )
        self.predictor._model = quantized_model
        self.model = self.predictor._model
        self.tokenizer = self.predictor._dataset_reader._tokenizer

    def __call__(self, text_input_list):
        outputs = []
        for text_input in text_input_list:
            outputs.append(self.predictor.predict(sentence=text_input))
        # For each output, outputs['logits'] contains the logits where
        # index 0 corresponds to the positive and index 1 corresponds
        # to the negative score. We reverse the outputs (by reverse slicing,
        # [::-1]) so that negative comes first and positive comes second.
        return [output['logits'][::-1] for output in outputs]
model_wrapper = AllenNLPModel()
model_wrapper_compressed = AllenNLPModelCompressed()

In [ ]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")
attack = TextFoolerJin2019.build(model_wrapper_compressed)
# attack = DeepWordBugGao2018.build(model_wrapper)

# attack_args = textattack.AttackArgs(
#     num_examples=500,
#     random_seed = 42,
#     query_budget = 80,
#     log_to_csv ="quantized_textfooler_attacks.csv"
# )
# attacker = Attacker(attack, dataset,attack_args)
# res_quant = attacker.attack_dataset()

In [4]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")
attack = TextFoolerJin2019.build(model_wrapper)
# attack = DeepWordBugGao2018.build(model_wrapper)

attack_args = textattack.AttackArgs(
    num_examples=500,
    random_seed = 42,
    query_budget = 80,
    log_to_csv ="main_textfooler_attacks.csv"
)
attacker = Attacker(attack, dataset,attack_args)
res_main = attacker.attack_dataset()

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset glue, subset sst2, split validation.
textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path main_textfooler_attacks.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 1/500 [00:08<1:08:44,  8.26s/it]/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|          | 1/500 [00:08<1:08:57,  8.29s/it]

...

--------------------------------------------- Result 498 ---------------------------------------------
[[Negative (94%)]] --> [[Positive (53%)]]

the overall [[effect]] is [[less]] like a children 's movie than a recruitment film for future hollywood sellouts . 

the overall [[affecting]] is [[minder]] like a children 's movie than a recruitment film for future hollywood sellouts . 




[Succeeded / Failed / Skipped / Total] 295 / 131 / 72 / 498: 100%|█████████▉| 499/500 [1:46:29<00:12, 12.80s/it]/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 296 / 131 / 72 / 499: 100%|█████████▉| 499/500 [1:46:29<00:12, 12.80s/it]

--------------------------------------------- Result 499 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (95%)]]

[[anchored]] by friel and williams 's [[exceptional]] performances , the film 's power lies in its complexity . 

[[pinned]] by friel and williams 's [[unpaid]] performances , the film 's power lies in its complexity . 




[Succeeded / Failed / Skipped / Total] 296 / 131 / 72 / 499: 100%|██████████| 500/500 [1:46:53<00:00, 12.83s/it]/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 297 / 131 / 72 / 500: 100%|██████████| 500/500 [1:46:53<00:00, 12.83s/it]

--------------------------------------------- Result 500 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (91%)]]

unlike the [[speedy]] wham-bam effect of most hollywood offerings , character development -- and more importantly , character empathy -- is at the [[heart]] of italian for beginners . 

unlike the [[hurry]] wham-bam effect of most hollywood offerings , character development -- and more importantly , character empathy -- is at the [[cardiology]] of italian for beginners . 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 297    |
| Number of failed attacks:     | 131    |
| Number of skipped attacks:    | 72     |
| Original accuracy:            | 85.6%  |
| Accuracy under attack:        | 26.2%  |
| Attack success rate:          | 69.39% |
| Average perturbed word %:     | 13.64% |
| Average num. words per input: | 17.5   |
| 

In [33]:
res

In [34]:
res[1].original_text() , res[1].perturbed_text()

('unflinchingly bleak and desperate ', 'unflinchingly Vleak and Mdesperate ')

In [40]:
attack.attack("unflinchingly Vleak and Mdesperate ", 1)

In [3]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")
attack = TextFoolerJin2019.build(model_wrapper)
attack_comp = TextFoolerJin2019.build(model_wrapper_compressed)

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset glue, subset sst2, split validation.
textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Unknown if model of class <class 'allennlp.models.basic_classifier.BasicClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [4]:
import pandas as pd
main_df = pd.read_csv('main_textfooler_attacks.csv')
quant_df = pd.read_csv('quantized_textfooler_attacks.csv')

In [ ]:
quant_df

In [7]:
main_atts = [x for x in main_df['perturbed_text'].tolist()[:100]]
quant_atts = [x for x in quant_df['perturbed_text'].tolist()[:100]]
labs = [x for x in main_df['ground_truth_output'].tolist()[:100]]

In [8]:
res = []
for i,x in enumerate(main_atts):
    res.append(attack_comp.attack(x, int(labs[i])))

2022-05-15 05:05:18.809565: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-15 05:05:18.847061: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-15 05:05:18.847103: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gv015.hpc.nyu.edu
2022-05-15 05:05:18.847112: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gv015.hpc.nyu.edu
2022-05-15 05:05:18.847302: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Invalid argument: expected %d.%d, %d.%d.%d, or %d.%d.%d.%d form for driver version; got "1"
2022-05-15 05:05:18.847333: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.57.2
2022-05-15 05:05:18.847749: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow

In [9]:
res_q = []
for i,x in enumerate(quant_atts):
    res_q.append(attack.attack(x, int(labs[i])))